In [1]:
# import packages
import numpy as np
from numpy import random
from scipy import stats
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split


In [2]:
data = pd.read_csv('/Users/arberimbibaj/dataset_example_indicatorCATE.csv', header=None, index_col=[0])
data = data.to_numpy()
data

array([[  4.907737,  -0.23949 ,   0.202046, ...,  -0.274776,   1.      ,
          8.      ],
       [-15.366406,   0.785973,  -0.935212, ...,   0.146119,   1.      ,
          0.      ],
       [  8.007676,   1.097277,  -0.962872, ...,  -0.791275,   1.      ,
          0.      ],
       ...,
       [ 14.967829,   1.437212,   0.055816, ...,   0.720359,   0.      ,
          0.      ],
       [  8.21161 ,  -0.568252,   0.788217, ...,   0.234121,   0.      ,
          8.      ],
       [ -4.206514,  -0.792531,   1.636563, ...,   2.661297,   0.      ,
          8.      ]])

In [3]:
N = len(data)
d = len(data[0,:]) - 3
print(N)
print(d)

1000
25


In [4]:
# train test split
random.shuffle(data)
training, test = data[:700,:], data[700:,:]

In [5]:
# slice dataset by treatment status
training_control = training[training[:,26]==0]
training_treatment = training[training[:,26]==1]

# Y_train by treatment status
Y_train_control = training_control[:,0]
Y_train_treatment = training_treatment[:,0]

# X_train by treatment status
X_train_control = training_control[:,1:26]
X_train_treatment = training_treatment[:,1:26]

# X and Y test
X_test = test[:,1:26]
Y_test = test[:,0]

# X_train and Y_train (no split by treatment status)
X_train = training[:,1:26]
Y_train = training[:,0]

# W_train and W_test
W_train = training[:,26]
W_test = test[:,26]

# tau_test
tau_test = test[:,27]

In [ ]:
X_train_control

In [ ]:
X_train_treatment

In [ ]:
Y_train_control

In [ ]:
Y_train_treatment

In [ ]:
X_train

In [ ]:
W_train

In [ ]:
Y_train

In [ ]:
X_test

In [ ]:
W_test

In [ ]:
Y_test

In [ ]:
tau_test

# T-Learner

In [6]:
# T-Learner (example with Random Forest)

# mu_0
t_learner_mu0 = RandomForestRegressor(max_depth=100, random_state=0)
t_learner_mu0.fit(X_train_control,Y_train_control)
t_mu_0_hat = t_learner_mu0.predict(X_test)

# mu_1
t_learner_mu1 = RandomForestRegressor(max_depth=100, random_state=0)
t_learner_mu1.fit(X_train_treatment,Y_train_treatment)
t_mu_1_hat = t_learner_mu1.predict(X_test)
# Prediction = mu_1 - mu_0
t_tau_hat = t_mu_1_hat - t_mu_0_hat
t_tau_hat

array([ 7.93023299,  7.57363402,  6.28923954,  6.35605731,  4.93571804,
        5.7948198 ,  6.15786102,  5.69702577, -3.23607085, -0.82471495,
       -0.02757093, -1.97234938, -2.0989381 ,  9.99820911,  8.07374076,
        0.48530611,  3.25043007,  9.10631944,  6.43556266,  1.39081276,
       -1.35001487,  8.88627439,  3.62163239,  7.95661281, -1.49176615,
        9.51605195, -3.86173957,  0.27736105,  4.50266117,  5.8748532 ,
        6.81967138,  8.54330084,  0.60840912, -0.89707725, 13.68530122,
        8.10353288, -1.7086585 ,  7.12674604, -6.38876342, 10.11365017,
        6.31753042,  5.30402283,  0.24191387,  9.42889418, -1.66766678,
        5.72901663, -1.53302361,  7.80419692,  1.21674555,  0.62269069,
        7.53613745, -1.53422593,  7.47987102, -3.61006324, -2.25618583,
        8.55823157,  7.73002624, -4.58249584, 12.18705093, -1.13550755,
       -0.5510779 ,  6.95458865, 13.82171131,  8.80664569, -0.51759794,
        4.77105314, 11.56175174,  3.76406674,  0.37337089,  8.60

In [7]:
# mean squared error
((t_tau_hat - tau_test)**2).mean()

9.776528238727845

# S-Learner

In [8]:
### S-Learner

In [9]:
# set training and test features for the S-Learner (it views W as no different from other X's)
X_W_train = training[:,1:27]
X_test_0 = np.concatenate((test[:,1:26],np.zeros((300,1))), axis=1)
X_test_1 = np.concatenate((test[:,1:26],np.ones((300,1))), axis=1)

In [10]:
X_test_0

array([[-1.574516,  0.816348, -0.403213, ..., -0.075188, -0.994311,
         0.      ],
       [ 1.213255,  0.57624 , -1.393691, ...,  0.956216,  0.557876,
         0.      ],
       [ 0.960647, -0.146155, -0.144904, ...,  0.623523,  0.185637,
         0.      ],
       ...,
       [-1.252195,  0.999406, -0.474349, ..., -0.892146,  0.443439,
         0.      ],
       [ 1.064068, -0.975303,  1.023323, ...,  0.342776, -0.043484,
         0.      ],
       [-0.599475,  1.234129, -1.210025, ..., -0.621704,  1.105705,
         0.      ]])

In [ ]:
X_test_1

In [ ]:
Y_train

In [11]:
# S-learner (example with Random Forest)

# mu_x
s_learner = RandomForestRegressor(max_depth=100, random_state=0)
s_learner.fit(X_W_train,Y_train)

# mu_0_hat
s_mu_0_hat = s_learner.predict(X_test_0)

# mu_1_hat
s_mu_1_hat = s_learner.predict(X_test_1)

# tau_hat
s_tau_hat = s_mu_1_hat - s_mu_0_hat
s_tau_hat


array([ 1.97634476e+00,  2.89353131e+00,  1.46136843e+00,  4.03093150e-01,
        1.95380100e-02,  1.81968405e+00,  3.06581395e+00,  4.50158146e+00,
       -5.11393300e-02,  7.34489000e-03,  1.19505300e-02, -1.68233880e-01,
        3.66830500e-01,  7.20921981e+00,  2.14613702e+00,  2.12366762e+00,
        9.58768620e-01,  1.13519120e+00,  2.85044540e+00,  8.07172200e-02,
       -1.06239940e-01,  5.25327446e+00,  3.06628940e-01,  9.56450440e-01,
        4.72788500e-01,  6.61685460e+00,  2.37685060e-01, -2.22887300e-02,
        1.90801134e+00,  1.59052775e+00,  3.66315925e+00,  1.21623626e+00,
        1.28833576e+00, -1.24915900e-02,  2.95162159e+00,  1.53514266e+00,
       -7.95322000e-02,  2.38599703e+00, -1.21904040e-01,  4.87242643e+00,
        2.35312921e+00,  1.29383692e+00, -2.78649600e-02,  1.90205857e+00,
        2.11607720e-01,  2.55849781e+00,  5.14945700e-02,  5.21077510e-01,
       -1.09514450e-01, -5.92234600e-02,  2.24598710e-01,  1.01347585e+00,
        2.69627560e-01, -

In [12]:
# mean squared error
((s_tau_hat - tau_test)**2).mean()

13.296254060893528

# X-Learner

In [13]:
### X-Learner

# mu_0 (same procedure as for t-learner, maybe can speed up process)
x_learner_mu0 = RandomForestRegressor(max_depth=100, random_state=0)
x_learner_mu0.fit(X_train_control,Y_train_control)

# mu_1 (same procedure as for t-learner, maybe can speed up process)
x_learner_mu1 = RandomForestRegressor(max_depth=100, random_state=0)
x_learner_mu1.fit(X_train_treatment,Y_train_treatment)

# compute imputed treatment effect D_0 and D_1
# d_0
imputed_0 = x_learner_mu1.predict(X_train_control) - Y_train_control

# d_1
imputed_1 = Y_train_treatment - x_learner_mu0.predict(X_train_treatment)

# regress imputed on X
# tau_hat_0
x_tau_0_hat = RandomForestRegressor(max_depth=100, random_state=0)
x_tau_0_hat.fit(X_train_control ,imputed_0)

# tau_hat_1
x_tau_1_hat = RandomForestRegressor(max_depth=100, random_state=0)
x_tau_1_hat.fit(X_train_treatment ,imputed_1)

# estimate e_x to use as g_x
g_x_hat = RandomForestClassifier(max_depth=100, random_state=0)
g_x_hat.fit(X_train,W_train)
probabilities = g_x_hat.predict_proba(X_test)
probas_1 = probabilities[:,1]
probas_0 = probabilities[:,0]

# final estimator of tau
x_tau_hat = probas_1 * x_tau_0_hat.predict(X_test) + probas_0 * x_tau_1_hat.predict(X_test)
x_tau_hat


array([ 9.72090454e+00,  7.72075506e+00,  2.26596069e+00,  3.68644191e+00,
        1.42361294e+00,  8.37483401e+00,  7.78210372e+00,  7.67671254e+00,
        2.34393186e-01, -1.77336932e-01, -2.83804969e+00, -3.54085684e+00,
       -1.10821831e+00,  6.81998009e+00,  6.25449633e+00,  9.85038400e+00,
        1.86785211e+00,  7.97610823e+00,  5.92764210e+00, -7.24100680e+00,
        2.99772705e-01,  7.76440617e+00,  3.97696500e-01,  7.78121200e+00,
        2.85456626e+00,  7.45531873e+00,  1.22383155e+00, -9.02763874e-01,
        3.46404331e-01,  6.27789211e+00,  7.70509469e+00,  8.39261531e+00,
        2.51122989e+00,  2.31188256e-01,  8.35889976e+00,  8.35493647e+00,
       -5.15406656e-01,  7.34248495e+00, -2.77527667e+00,  6.85416639e+00,
        5.11979617e+00,  8.97470202e+00,  3.06116562e-01,  8.80122540e+00,
        2.38236451e+00,  8.98057942e+00,  1.50264375e+00,  7.74871129e+00,
       -5.50078056e-01,  5.37235297e-01,  8.30886116e+00,  2.94092431e+00,
        6.11359799e+00, -

In [14]:
# mean squared error (much lower here!)
((x_tau_hat - tau_test)**2).mean()

3.603789312509816

# R-Learner

In [36]:
### R-Learner

# estimate e_x
r_learner_e_x = RandomForestClassifier(max_depth=100, random_state=0)
r_learner_e_x.fit(X_train,W_train)

# get e_x predictions
r_probas = r_learner_e_x.predict_proba(X_train)
r_probas_0 = r_probas[:,0] # probabilities of W=0
r_probas_1 = r_probas[:,1] # probabilities of W=1

# estimate mu_x
r_learner_mu_x = RandomForestRegressor(max_depth=100, random_state=0)
r_learner_mu_x.fit(X_train,Y_train)

# compute r-pseudo-outcome and weights
r_learner_pseudo_outcomes = (Y_train - r_learner_mu_x.predict(X_train)) / (W_train - r_probas_1)
r_learner_weights = (W_train - r_probas_1)**2

# estimate tau (regress pseudo-outcomes on X, weight by (W-e(x))^2)
r_learner_tau = RandomForestRegressor(max_depth=100, random_state=0)
r_learner_tau.fit(X_train,r_learner_pseudo_outcomes, sample_weight=r_learner_weights)

# predict tau
r_tau_hats = r_learner_tau.predict(X_test)
r_tau_hats

array([ 16.65314113,   1.85655957,  -0.09162369,   1.49776494,
         3.32048113,   3.41824568,   9.04369837,   6.88941744,
         3.60283165,   0.27407187,   1.16309896,  -1.28681172,
        -9.97846686,  11.32064081,   8.6071461 ,   4.66830824,
        -3.41801481,   9.40964865,   7.28798145,  -6.58603365,
        -1.02770548,   5.97938502,   2.46005614,   9.0578432 ,
       -17.10626898,   8.18943018,  -1.83491241,  -1.19833961,
         0.83462304,   4.86463687,  12.29791947,   6.40089777,
         4.58989048,  -1.56027934,  11.36879664,   7.39107064,
         1.29784705,   6.55156421,  -3.54200553,   7.99604272,
        10.77088109,  11.30839091,  -7.92437334,  12.30285498,
         1.66344972,  13.45231909,   2.22140779,   6.22614468,
         1.18442984,   0.20920783,  12.74558585,  -1.43276476,
         4.87960468,  -1.84614308,  -3.40273126,  10.57487176,
        -2.56862785,   0.40300404,   5.38408702,   2.28014517,
         2.25106512,   9.8482129 ,  10.01712472,   8.89

In [37]:
((r_tau_hats - tau_test)**2).mean()

20.924340579229042

# DR-Learner

In [26]:
### DR-Learner

# TODO: APPLY CROSS-FITTING?
# estimate e_x
dr_learner_e_x = RandomForestClassifier(max_depth=100, random_state=0)
dr_learner_e_x.fit(X_train, W_train)

dr_probas = dr_learner_e_x.predict_proba(X_train)
dr_probas_0 = dr_probas[:,0] # probabilities of W=0
dr_probas_1 = dr_probas[:,1] # probabilities of W=1

# estimate mu_0
dr_learner_mu_0 = RandomForestRegressor(max_depth=100, random_state=0)
dr_learner_mu_0.fit(X_train_control,Y_train_control)

# estimate mu_1
dr_learner_mu_1 = RandomForestRegressor(max_depth=100, random_state=0)
dr_learner_mu_1.fit(X_train_treatment,Y_train_treatment)

# DR-pseudo-outcomes
mu_w = W_train * dr_learner_mu_1.predict(X_train) + (1 - W_train) * dr_learner_mu_0.predict(X_train) # this is mu_w for each observation, i.e mu_1 for units in the treatment groups, and mu_0 for units in the control group
dr_pseudo_outcomes = (W_train - dr_probas_1) / (dr_probas_1 * dr_probas_0) * (Y_train - mu_w) + dr_learner_mu_1.predict(X_train) - dr_learner_mu_0.predict(X_train)

# estimate tau (regress pseudo-outcomes on X) # TODO: USE "Test Set" for this estimation
dr_learner_tau_hat = RandomForestRegressor(max_depth=100, random_state=0)
dr_learner_tau_hat.fit(X_train,dr_pseudo_outcomes)

# predict tau
dr_tau_hat = dr_learner_tau_hat.predict(X_test)
dr_tau_hat

array([ 1.10878897e+01,  8.45519934e+00,  2.09613951e+00,  1.98497439e+00,
        2.68505935e+00,  3.88907801e+00,  9.81805232e+00,  7.57391955e+00,
        1.20804460e+00,  2.19286265e-01,  1.15463542e+00, -9.88651781e-01,
       -2.74085129e+00,  1.00943375e+01,  7.90568968e+00,  4.62178932e+00,
        7.19823635e+00,  7.95476085e+00,  8.62034820e+00, -2.07416576e+00,
       -5.38395047e-01,  8.97372291e+00,  1.48805013e+00,  5.38461017e+00,
       -4.23267109e+00,  8.65418005e+00, -4.44779810e-02, -8.09128322e-01,
        4.21997423e+00,  3.87024089e+00,  9.91297658e+00,  9.10635285e+00,
        2.11012866e+00,  8.31239248e-01,  9.26997462e+00,  7.34651334e+00,
        1.44173951e+00,  8.52486800e+00, -2.45020527e+00,  7.43438592e+00,
        5.98707121e+00,  7.92205012e+00, -3.09560436e+00,  1.13998392e+01,
        2.03325931e+00,  9.00474961e+00,  5.06144870e-01,  7.50434212e+00,
        4.01893959e-01,  8.35290846e-02,  8.23007240e+00,  5.89472740e-01,
        1.72832322e+00, -

In [27]:
((dr_tau_hat - tau_test)**2).mean()

5.231521628057535

# RA-Learner

In [17]:
### RA-Learner

# mu_0 (same procedure as for t-learner, maybe can speed up process)
ra_learner_mu0 = RandomForestRegressor(max_depth=100, random_state=0)
ra_learner_mu0.fit(X_train_control,Y_train_control)

# mu_1 (same procedure as for t-learner, maybe can speed up process)
ra_learner_mu1 = RandomForestRegressor(max_depth=100, random_state=0)
ra_learner_mu1.fit(X_train_treatment,Y_train_treatment)

# e_x
ra_learner_e_x = RandomForestClassifier(max_depth=100, random_state=0)
ra_learner_e_x.fit(X_train,W_train)

# ra-pseudo-outcome
ra_pseudo_outcome = W_train*(Y_train - ra_learner_mu0.predict(X_train)) + (1 - W_train)*(ra_learner_mu1.predict(X_train) - Y_train)

# tau_hat
ra_tau_hat_learner = RandomForestRegressor(max_depth=100, random_state=0)
ra_tau_hat_learner.fit(X_train, ra_pseudo_outcome)
ra_tau_hat = ra_tau_hat_learner.predict(X_test)
ra_tau_hat

array([ 1.02928281e+01,  7.52823953e+00,  1.83310868e+00,  1.92102385e+00,
        1.97682589e+00,  3.93716774e+00,  1.02442345e+01,  7.23637949e+00,
        5.48833381e-02,  7.21022868e-01,  1.06170182e+00, -1.34752849e+00,
       -2.89046576e+00,  9.51428759e+00,  8.09291831e+00,  6.66119208e+00,
        7.31243876e+00,  7.79465383e+00,  8.84535573e+00, -2.06527121e+00,
       -7.37287667e-01,  9.58660920e+00,  8.06455025e-01,  5.70022858e+00,
       -4.10375743e+00,  8.16392674e+00, -6.29653142e-01, -1.10465179e+00,
        5.05753377e+00,  3.63015816e+00,  9.98372064e+00,  9.05465614e+00,
        2.87604889e+00,  6.82915218e-01,  8.82625090e+00,  7.13804780e+00,
        1.26766775e+00,  8.23419207e+00, -2.99954248e+00,  7.48444745e+00,
        6.14995163e+00,  8.06522761e+00, -2.88047138e+00,  1.12055193e+01,
        1.69367981e+00,  9.36835013e+00,  4.21616812e-01,  7.15074452e+00,
       -2.80823635e-01,  4.00562236e-01,  8.42738634e+00, -5.48132348e-01,
        1.37239983e+00, -

In [18]:
# mean squared error
((ra_tau_hat - tau_test)**2).mean()

5.138584982993711

# PW-Learner

In [19]:
### PW-Learner
# mu_0 (same procedure as for t-learner, maybe can speed up process)
pw_learner_mu0 = RandomForestRegressor(max_depth=100, random_state=0)
pw_learner_mu0.fit(X_train_control,Y_train_control)

# mu_1 (same procedure as for t-learner, maybe can speed up process)
pw_learner_mu1 = RandomForestRegressor(max_depth=100, random_state=0)
pw_learner_mu1.fit(X_train_treatment,Y_train_treatment)

# e_x
pw_learner_e_x = RandomForestClassifier(max_depth=100, random_state=0)
pw_learner_e_x.fit(X_train,W_train)

# ra-pseudo-outcome
pw_pseudo_outcome = (W_train/pw_learner_e_x.predict_proba(X_train)[:,1] - (1 - W_train)/(pw_learner_e_x.predict_proba(X_train)[:,0]))*Y_train

# tau_hat
pw_tau_hat_learner = RandomForestRegressor(max_depth=100, random_state=0)
pw_tau_hat_learner.fit(X_train, pw_pseudo_outcome)
pw_tau_hat = pw_tau_hat_learner.predict(X_test)
pw_tau_hat

array([ 6.41799953e+00, -1.21020971e+01,  6.00846327e-01,  1.19072386e+00,
        3.81170965e+00,  7.43249389e+00,  1.63750732e+00,  5.56722277e+00,
       -1.21833171e+00, -9.70121739e-01, -5.85972584e+00,  2.40906320e+00,
       -1.01717035e+00,  8.06051345e+00,  2.18066164e+01, -8.02036196e+00,
       -9.53632528e+00,  5.13667869e+00,  3.74617629e+00,  1.45405398e+00,
        1.09174448e-01,  9.55698606e+00, -2.09472718e+00,  6.79020887e+00,
       -2.99854635e+00,  2.55581732e+00, -1.13011682e+00, -3.40207697e+00,
       -8.82599784e+00,  1.05666924e+01,  7.24991092e+00,  1.98337416e+01,
        3.08081394e+00, -2.01722383e+00,  5.71946894e+00,  8.83048288e-01,
        2.11229022e+00,  5.48203131e+00,  2.66681590e+00,  5.57886349e+00,
        1.10329492e+01,  3.31235795e+00, -1.63875949e+00,  2.04758083e+01,
        3.78401242e+00,  6.92490082e+00, -6.21339613e-01,  7.08225147e+00,
        1.25708922e+00,  1.96116059e+00,  8.52859218e+00,  6.69037597e-01,
        4.45417139e+00, -

In [20]:
# mean squared error
((pw_tau_hat - tau_test)**2).mean()

30.27374090972972

# F-Learner: AKA THE SAME AS PW-LEARNER!!!

In [21]:
### F-Learner
# mu_0 (same procedure as for t-learner, maybe can speed up process)
f_learner_mu0 = RandomForestRegressor(max_depth=100, random_state=0)
f_learner_mu0.fit(X_train_control,Y_train_control)

# mu_1 (same procedure as for t-learner, maybe can speed up process)
f_learner_mu1 = RandomForestRegressor(max_depth=100, random_state=0)
f_learner_mu1.fit(X_train_treatment,Y_train_treatment)

# e_x
f_learner_e_x = RandomForestClassifier(max_depth=100, random_state=0)
f_learner_e_x.fit(X_train,W_train)

# ra-pseudo-outcome
f_pseudo_outcome = (W_train/f_learner_e_x.predict_proba(X_train)[:,1] - (1 - W_train)/(f_learner_e_x.predict_proba(X_train)[:,0]))*Y_train

# tau_hat
f_tau_hat_learner = RandomForestRegressor(max_depth=100, random_state=0)
f_tau_hat_learner.fit(X_train, pw_pseudo_outcome)
f_tau_hat = f_tau_hat_learner.predict(X_test)
f_tau_hat

array([ 6.41799953e+00, -1.21020971e+01,  6.00846327e-01,  1.19072386e+00,
        3.81170965e+00,  7.43249389e+00,  1.63750732e+00,  5.56722277e+00,
       -1.21833171e+00, -9.70121739e-01, -5.85972584e+00,  2.40906320e+00,
       -1.01717035e+00,  8.06051345e+00,  2.18066164e+01, -8.02036196e+00,
       -9.53632528e+00,  5.13667869e+00,  3.74617629e+00,  1.45405398e+00,
        1.09174448e-01,  9.55698606e+00, -2.09472718e+00,  6.79020887e+00,
       -2.99854635e+00,  2.55581732e+00, -1.13011682e+00, -3.40207697e+00,
       -8.82599784e+00,  1.05666924e+01,  7.24991092e+00,  1.98337416e+01,
        3.08081394e+00, -2.01722383e+00,  5.71946894e+00,  8.83048288e-01,
        2.11229022e+00,  5.48203131e+00,  2.66681590e+00,  5.57886349e+00,
        1.10329492e+01,  3.31235795e+00, -1.63875949e+00,  2.04758083e+01,
        3.78401242e+00,  6.92490082e+00, -6.21339613e-01,  7.08225147e+00,
        1.25708922e+00,  1.96116059e+00,  8.52859218e+00,  6.69037597e-01,
        4.45417139e+00, -

In [22]:
# mean squared error
print(((f_tau_hat - tau_test)**2).mean())
print("Same as for PW-Learner")

30.27374090972972
Same as for PW-Learner


# U-Learner

In [23]:
### U-Learner
# estimate e_x
u_learner_e_x = RandomForestClassifier(max_depth=100, random_state=0)
u_learner_e_x.fit(X_train,W_train)

# estimate mu_x
u_learner_mu_x = RandomForestRegressor(max_depth=100, random_state=0)
u_learner_mu_x.fit(X_train,Y_train)

# compute residuals
u_learner_residuals = (Y_train - u_learner_mu_x.predict(X_train))/(W_train - u_learner_e_x.predict_proba(X_train)[:,1])

# tau_hat - regress residuals on X
u_tau_hat_learner = RandomForestRegressor(max_depth=100, random_state=0)
u_tau_hat_learner.fit(X_train,u_learner_residuals)

u_tau_hats = u_tau_hat_learner.predict(X_test)
u_tau_hats


array([ 1.53229471e+01,  4.47928319e+00,  1.64919834e+00,  4.85817306e-01,
        2.26181817e+00, -1.44698961e+00,  1.18130329e+01,  8.81277383e+00,
        2.78697192e+00,  1.85608519e+00,  9.78005991e-01,  4.59731357e-01,
       -4.90533123e+00,  1.06995224e+01,  1.03578494e+01,  7.63756635e+00,
       -9.36174451e+00,  1.06612263e+01,  6.11743844e+00, -1.07766639e+01,
       -1.10920992e+00,  6.46747519e+00,  1.94457463e+00,  1.69784065e+01,
       -2.22702601e+01,  7.83596153e+00,  2.85630366e-01, -3.70516998e+00,
        5.91209922e+00,  3.67813864e+00,  1.08086585e+01,  7.35971638e+00,
        1.40523060e+00, -1.31058185e-01,  1.15578606e+01,  7.87797829e+00,
        5.65731787e+00,  6.16619391e+00, -5.27468377e-01,  7.98187996e+00,
        9.87809950e+00,  1.16812401e+01, -8.16162477e+00,  1.34746069e+01,
        4.13595718e+00,  1.88433047e+01,  3.40873436e-01,  6.90682425e+00,
        1.21507327e+00, -4.53851639e-01,  1.02432341e+01,  5.55115481e+00,
        5.17052774e+00,  

In [24]:
# mean squared error
((u_tau_hats - tau_test)**2).mean()

30.65664176997332

# Just some lasso tests

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly_train = PolynomialFeatures(degree=4, interaction_only=False, include_bias=False)
X_poly_train = poly_train.fit_transform(X_train)
poly_test = PolynomialFeatures(degree=4, interaction_only=False, include_bias=False)
X_poly_test = poly_test.fit_transform(X_test)
X_poly_train

In [ ]:
from sklearn.linear_model import Lasso, LassoCV
lasso_poly = LassoCV(cv=10, random_state=0, tol=1e-2)

In [ ]:
import time

t0 = time.time()
lasso_poly.fit(X_poly_train,Y_train)

t1 = time.time()

total = t1-t0
print(f"{total:.4f} seconds")

In [ ]:
# degree 3: 10 seconds to fit (100% cpu)
# degree 4: 90 seconds to fit (100% cpu)

In [ ]:
y_predictions = lasso_poly.predict(X_poly_test)

In [ ]:
((y_predictions - Y_test)**2).mean()

# just some Neural Network test

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [ ]:
# make model
# 3 layers with 200 units (elu activation), 2 layers with 100 units (elu activations), 1 output layer (linear activation)
model = keras.Sequential([
    keras.Input(shape=(d,)),
    layers.Dense(units=200, activation="relu", name="layer1"),
    layers.Dense(units=200, activation="relu", name="layer2"),
    layers.Dense(units=200, activation="relu", name="layer3"),
    layers.Dense(units=100, activation="relu", name="layer4"),
    layers.Dense(units=100, activation="relu", name="layer5"),
    layers.Dense(units=1, activation="linear", name="layer6"),

], name="Dense_Neural_Network")
model.summary()

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(),  # Optimizer
    # Loss function to minimize
    loss=keras.losses.MeanSquaredError(),
    # List of metrics to monitor
    metrics=[keras.metrics.MeanSquaredError()],
)

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, start_from_epoch=0)

In [ ]:
print("Training Model")
training = model.fit(
    X_train,
    Y_train,
    batch_size=100,
    epochs=10,
    validation_data=(X_test, Y_test),
    callbacks=[callback] # include early stopping
)

In [ ]:
predictions

In [ ]:
print("Generate predictions for test samples")
predictions = np.reshape(model.predict(X_test),(300,))
print("predictions shape:", predictions.shape)
predictions

In [ ]:
results = model.evaluate(X_test, Y_test, batch_size=100)

In [ ]:
((predictions - Y_test)**2).mean()